In [1]:
# importing useful libs on the go
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold
from xgboost.sklearn import XGBClassifier
import math
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn import cross_validation
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import neighbors
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import collections

plt.style.use('fivethirtyeight') # Good looking plots
pd.set_option('display.max_columns', None) # Display any number of columns

% matplotlib inline

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,21,23,26,28,29,30,32,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,26,28,29,30,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
lis_drop = ['EEG_TAG','EEG_CLOSED', 'PM_FD_MON_02', 'PM_FD_MON_04', 'STMT_CON_DAE_ACTIVE_MON_01', 'STMT_CON_DAE_CLOSED_MON_01', 'MER_EMI_CLOSED_MON_01']

In [4]:
train.drop(lis_drop, axis=1, inplace=True)
test.drop(lis_drop, axis=1, inplace=True)

In [5]:
test_cust = test['CUSTOMER_ID']

In [6]:
train.head()

,CUSTOMER_ID,ACT_TYPE,GENDER,LEGAL_ENTITY,OCCUP_ALL_NEW,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,BL_TAG,TWL_TAG,PL_TAG,AL_CNC_TAG,AGRI_TAG,CE_TAG,CV_TAG,EDU_TAG,TL_TAG,OTHER_LOANS_TAG,LAP_TAG,LAS_TAG,BBG_TAG,HL_TAG,GL_TAG,AL_CLOSED,PL_CLOSED,BL_CLOSED,TWL_CLOSED,CV_CLOSED,LAP_CLOSED,LAS_CLOSED,BBG_CLOSED,GL_CLOSED,CC_HOLD_MON_03,CC_HOLD_MON_02,CC_HOLD_MON_01,CC_ACTIVE_MON_03,CC_ACTIVE_MON_02,CC_ACTIVE_MON_01,CC_HOLD,CC_ACTIVE,DC_HOLD,DC_ACTIVE_MON_03,DC_ACTIVE_MON_02,DC_ACTIVE_MON_01,DC_ACTIVE,DC_SPEND_MON_03,DC_SPEND_MON_02,DC_SPEND_MON_01,CC_SPEND_MON_04,CC_SPEND_MON_01,CC_SPEND_MON_02,CC_SPEND_MON_03,DC_SPEND_MON_04,PM_FD_MON_01,CR_LIMIT,STMT_MON_03,STMT_MON_02,STMT_MON_01,COC_ELIGIBLE,LAP_SCRUB_LIVE,LAS_SCRUB_LIVE,PL_SCRUB_LIVE,GL_SCRUB_LIVE,TWL_SCRUB_LIVE,BL_SCRUB_LIVE,CEL_SCRUB_LIVE,TL_SCRUB_LIVE,AL_SCRUB_LIVE,LAP_SCRUB_CLOSED,LAS_SCRUB_CLOSED,PL_SCRUB_CLOSED,GL_SCRUB_CLOSED,TWL_SCRUB_CLOSED,BL_SCRUB_CLOSED,CEL_SCRUB_CLOSED,TL_SCRUB_CLOSED,AL_SCRUB_CLOSED,MATURITY_GL,MATURITY_LAP,MATURITY_LAS,BTE_ACTIVE_MON_01,COC_ACTIVE_MON_01,PL2CC_ACTIVE_MON_01,MER_EMI_ACTIVE_MON_01,TRN_CON_DAE_ACTIVE_MON_01,BTE_CLOSED_MON_01,COC_CLOSED_MON_01,PL2CC_CLOSED_MON_01,TRN_CON_DAE_CLOSED_MON_01,EOP_BAL_MON_01,AMB_MON_04,AMB_MON_01,AMB_MON_02,AMB_MON_03,NB_MON_01_CNT,NB_MON_02_CNT,NB_MON_03_CNT,CDMAT_MON_01,CDMAT_MON_04,MF_MON_04,MF_MON_01,RD_MON_04,RD_MON_01,FD_MON_04,FD_MON_01,LI_MON_04,LI_MON_01,GI_MON_04,GI_MON_01,CC_TXN_MON_01,CC_SPEND_MON_06,CC_TXN_MON_06,CC_TXN_MON_03,CC_TXN_MON_02,CC_TXN_MON_04,CC_SPEND_MON_05,CC_TXN_MON_05,DC_TXN_MON_04,DC_TXN_MON_01,DC_SPEND_MON_06,DC_TXN_MON_06,DC_SPEND_MON_05,DC_TXN_MON_05,DC_TXN_MON_03,DC_TXN_MON_02,DC_SPEND_MON_04_ENT,DC_SPEND_MON_04_MED,DC_SPEND_MON_04_GRC,DC_SPEND_MON_04_HMD,DC_SPEND_MON_04_HBY,DC_SPEND_MON_04_CARE,DC_SPEND_MON_04_TRL,DC_SPEND_MON_04_RST,DC_SPEND_MON_04_JER,DC_SPEND_MON_04_HTL,DC_SPEND_MON_04_ATM,DC_SPEND_MON_03_ENT,DC_SPEND_MON_03_MED,DC_SPEND_MON_03_HMD,DC_SPEND_MON_03_HBY,DC_SPEND_MON_03_CARE,DC_SPEND_MON_03_TRL,DC_SPEND_MON_03_RST,DC_SPEND_MON_03_JER,DC_SPEND_MON_03_HTL,DC_SPEND_MON_03_ATM,DC_SPEND_MON_02_ENT,DC_SPEND_MON_02_MED,DC_SPEND_MON_02_HMD,DC_SPEND_MON_02_HBY,DC_SPEND_MON_02_CARE,DC_SPEND_MON_02_TRL,DC_SPEND_MON_02_RST,DC_SPEND_MON_02_JER,DC_SPEND_MON_02_HTL,DC_SPEND_MON_02_ATM,DC_SPEND_MON_01_ENT,DC_SPEND_MON_01_MED,DC_SPEND_MON_01_HMD,DC_SPEND_MON_01_HBY,DC_SPEND_MON_01_CARE,DC_SPEND_MON_01_TRL,DC_SPEND_MON_01_RST,DC_SPEND_MON_01_JER,DC_SPEND_MON_01_HTL,DC_SPEND_MON_01_ATM,DC_SPEND_MON_05_ENT,DC_SPEND_MON_05_MED,DC_SPEND_MON_05_HMD,DC_SPEND_MON_05_HBY,DC_SPEND_MON_05_CARE,DC_SPEND_MON_05_TRL,DC_SPEND_MON_05_RST,DC_SPEND_MON_05_JER,DC_SPEND_MON_05_HTL,DC_SPEND_MON_05_ATM,CC_SPEND_MON_04_ENT,CC_SPEND_MON_04_MED,CC_SPEND_MON_04_HMD,CC_SPEND_MON_04_HBY,CC_SPEND_MON_04_CARE,CC_SPEND_MON_04_TRL,CC_SPEND_MON_04_RST,CC_SPEND_MON_04_JER,CC_SPEND_MON_04_HTL,CC_SPEND_MON_04_ATM,CC_SPEND_MON_02_ENT,CC_SPEND_MON_02_MED,CC_SPEND_MON_02_HMD,CC_SPEND_MON_02_HBY,CC_SPEND_MON_02_CARE,CC_SPEND_MON_02_TRL,CC_SPEND_MON_02_RST,CC_SPEND_MON_02_JER,CC_SPEND_MON_02_HTL,CC_SPEND_MON_02_ATM,CC_SPEND_MON_01_ENT,CC_SPEND_MON_01_MED,CC_SPEND_MON_01_HMD,CC_SPEND_MON_01_HBY,CC_SPEND_MON_01_CARE,CC_SPEND_MON_01_TRL,CC_SPEND_MON_01_RST,CC_SPEND_MON_01_JER,CC_SPEND_MON_01_HTL,CC_SPEND_MON_01_ATM,FINANCE_MON_01,JOBS_MON_01,LIFESTYLE_MON_01,INMON_01KET_MON_01,TRAVEL_MON_01,CHANNEL_CLICK_DISP,DESIGNATION_FINAL,TOP14_CITY,TOP9_CITY,PL2CC_ELIGIBLE,D_AMT_L3_MON_06,C_AMT_L3_MON_06,C_COUNT_L3_MON_06,D_COUNT_L3_MON_06,MAX_C_AMT_L3_MON_06,D_AMT_L3_MON_05,C_AMT_L3_MON_05,C_COUNT_L3_MON_05,D_COUNT_L3_MON_05,MAX_C_AMT_L3_MON_05,D_AMT_L3_MON_04,C_AMT_L3_MON_04,C_COUNT_L3_MON_04,D_COUNT_L3_MON_04,MAX_C_AMT_L3_MON_04,D_AMT_L3_MON_03,C_AMT_L3_MON_03,C_COUNT_L3_MON_03,D_COUNT_L3_MON_03,MAX_C_AMT_L3_MON_03,D_AMT_L3_MON_02,C_AMT_L3_MON_02,C_COUNT_L3_MON_02,D_COUNT_L3_MON_02,MAX_C_AMT_L3_MON_02,D_AMT_L3_MON_01,C_AMT_L3_MON_01,C_COUNT_L3_MON_01,D_COUNT_L3_MON_01,MAX_C_AMT_L3_MON_01,PA_PQ_TAG,NEFT_CC_AMOUNT,N

In [7]:
test.head()

,CUSTOMER_ID,ACT_TYPE,GENDER,LEGAL_ENTITY,OCCUP_ALL_NEW,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,BL_TAG,TWL_TAG,PL_TAG,AL_CNC_TAG,AGRI_TAG,CE_TAG,CV_TAG,EDU_TAG,TL_TAG,OTHER_LOANS_TAG,LAP_TAG,LAS_TAG,BBG_TAG,HL_TAG,GL_TAG,AL_CLOSED,PL_CLOSED,BL_CLOSED,TWL_CLOSED,CV_CLOSED,LAP_CLOSED,LAS_CLOSED,BBG_CLOSED,GL_CLOSED,CC_HOLD_MON_03,CC_HOLD_MON_02,CC_HOLD_MON_01,CC_ACTIVE_MON_03,CC_ACTIVE_MON_02,CC_ACTIVE_MON_01,CC_HOLD,CC_ACTIVE,DC_HOLD,DC_ACTIVE_MON_03,DC_ACTIVE_MON_02,DC_ACTIVE_MON_01,DC_ACTIVE,DC_SPEND_MON_03,DC_SPEND_MON_02,DC_SPEND_MON_01,CC_SPEND_MON_04,CC_SPEND_MON_01,CC_SPEND_MON_02,CC_SPEND_MON_03,DC_SPEND_MON_04,PM_FD_MON_01,CR_LIMIT,STMT_MON_03,STMT_MON_02,STMT_MON_01,COC_ELIGIBLE,LAP_SCRUB_LIVE,LAS_SCRUB_LIVE,PL_SCRUB_LIVE,GL_SCRUB_LIVE,TWL_SCRUB_LIVE,BL_SCRUB_LIVE,CEL_SCRUB_LIVE,TL_SCRUB_LIVE,AL_SCRUB_LIVE,LAP_SCRUB_CLOSED,LAS_SCRUB_CLOSED,PL_SCRUB_CLOSED,GL_SCRUB_CLOSED,TWL_SCRUB_CLOSED,BL_SCRUB_CLOSED,CEL_SCRUB_CLOSED,TL_SCRUB_CLOSED,AL_SCRUB_CLOSED,MATURITY_GL,MATURITY_LAP,MATURITY_LAS,BTE_ACTIVE_MON_01,COC_ACTIVE_MON_01,PL2CC_ACTIVE_MON_01,MER_EMI_ACTIVE_MON_01,TRN_CON_DAE_ACTIVE_MON_01,BTE_CLOSED_MON_01,COC_CLOSED_MON_01,PL2CC_CLOSED_MON_01,TRN_CON_DAE_CLOSED_MON_01,EOP_BAL_MON_01,AMB_MON_04,AMB_MON_01,AMB_MON_02,AMB_MON_03,NB_MON_01_CNT,NB_MON_02_CNT,NB_MON_03_CNT,CDMAT_MON_01,CDMAT_MON_04,MF_MON_04,MF_MON_01,RD_MON_04,RD_MON_01,FD_MON_04,FD_MON_01,LI_MON_04,LI_MON_01,GI_MON_04,GI_MON_01,CC_TXN_MON_01,CC_SPEND_MON_06,CC_TXN_MON_06,CC_TXN_MON_03,CC_TXN_MON_02,CC_TXN_MON_04,CC_SPEND_MON_05,CC_TXN_MON_05,DC_TXN_MON_04,DC_TXN_MON_01,DC_SPEND_MON_06,DC_TXN_MON_06,DC_SPEND_MON_05,DC_TXN_MON_05,DC_TXN_MON_03,DC_TXN_MON_02,DC_SPEND_MON_04_ENT,DC_SPEND_MON_04_MED,DC_SPEND_MON_04_GRC,DC_SPEND_MON_04_HMD,DC_SPEND_MON_04_HBY,DC_SPEND_MON_04_CARE,DC_SPEND_MON_04_TRL,DC_SPEND_MON_04_RST,DC_SPEND_MON_04_JER,DC_SPEND_MON_04_HTL,DC_SPEND_MON_04_ATM,DC_SPEND_MON_03_ENT,DC_SPEND_MON_03_MED,DC_SPEND_MON_03_HMD,DC_SPEND_MON_03_HBY,DC_SPEND_MON_03_CARE,DC_SPEND_MON_03_TRL,DC_SPEND_MON_03_RST,DC_SPEND_MON_03_JER,DC_SPEND_MON_03_HTL,DC_SPEND_MON_03_ATM,DC_SPEND_MON_02_ENT,DC_SPEND_MON_02_MED,DC_SPEND_MON_02_HMD,DC_SPEND_MON_02_HBY,DC_SPEND_MON_02_CARE,DC_SPEND_MON_02_TRL,DC_SPEND_MON_02_RST,DC_SPEND_MON_02_JER,DC_SPEND_MON_02_HTL,DC_SPEND_MON_02_ATM,DC_SPEND_MON_01_ENT,DC_SPEND_MON_01_MED,DC_SPEND_MON_01_HMD,DC_SPEND_MON_01_HBY,DC_SPEND_MON_01_CARE,DC_SPEND_MON_01_TRL,DC_SPEND_MON_01_RST,DC_SPEND_MON_01_JER,DC_SPEND_MON_01_HTL,DC_SPEND_MON_01_ATM,DC_SPEND_MON_05_ENT,DC_SPEND_MON_05_MED,DC_SPEND_MON_05_HMD,DC_SPEND_MON_05_HBY,DC_SPEND_MON_05_CARE,DC_SPEND_MON_05_TRL,DC_SPEND_MON_05_RST,DC_SPEND_MON_05_JER,DC_SPEND_MON_05_HTL,DC_SPEND_MON_05_ATM,CC_SPEND_MON_04_ENT,CC_SPEND_MON_04_MED,CC_SPEND_MON_04_HMD,CC_SPEND_MON_04_HBY,CC_SPEND_MON_04_CARE,CC_SPEND_MON_04_TRL,CC_SPEND_MON_04_RST,CC_SPEND_MON_04_JER,CC_SPEND_MON_04_HTL,CC_SPEND_MON_04_ATM,CC_SPEND_MON_02_ENT,CC_SPEND_MON_02_MED,CC_SPEND_MON_02_HMD,CC_SPEND_MON_02_HBY,CC_SPEND_MON_02_CARE,CC_SPEND_MON_02_TRL,CC_SPEND_MON_02_RST,CC_SPEND_MON_02_JER,CC_SPEND_MON_02_HTL,CC_SPEND_MON_02_ATM,CC_SPEND_MON_01_ENT,CC_SPEND_MON_01_MED,CC_SPEND_MON_01_HMD,CC_SPEND_MON_01_HBY,CC_SPEND_MON_01_CARE,CC_SPEND_MON_01_TRL,CC_SPEND_MON_01_RST,CC_SPEND_MON_01_JER,CC_SPEND_MON_01_HTL,CC_SPEND_MON_01_ATM,FINANCE_MON_01,JOBS_MON_01,LIFESTYLE_MON_01,INMON_01KET_MON_01,TRAVEL_MON_01,CHANNEL_CLICK_DISP,DESIGNATION_FINAL,TOP14_CITY,TOP9_CITY,PL2CC_ELIGIBLE,D_AMT_L3_MON_06,C_AMT_L3_MON_06,C_COUNT_L3_MON_06,D_COUNT_L3_MON_06,MAX_C_AMT_L3_MON_06,D_AMT_L3_MON_05,C_AMT_L3_MON_05,C_COUNT_L3_MON_05,D_COUNT_L3_MON_05,MAX_C_AMT_L3_MON_05,D_AMT_L3_MON_04,C_AMT_L3_MON_04,C_COUNT_L3_MON_04,D_COUNT_L3_MON_04,MAX_C_AMT_L3_MON_04,D_AMT_L3_MON_03,C_AMT_L3_MON_03,C_COUNT_L3_MON_03,D_COUNT_L3_MON_03,MAX_C_AMT_L3_MON_03,D_AMT_L3_MON_02,C_AMT_L3_MON_02,C_COUNT_L3_MON_02,D_COUNT_L3_MON_02,MAX_C_AMT_L3_MON_02,D_AMT_L3_MON_01,C_AMT_L3_MON_01,C_COUNT_L3_MON_01,D_COUNT_L3_MON_01,MAX_C_AMT_L3_MON_01,PA_PQ_TAG,NEFT_CC_AMOUNT,N

In [8]:
(train['DC_SPEND_MON_05'] == 2200).astype(int).sum(axis=0)

63

In [9]:
#zero_containing_columns   #51 cols
for col in list(train):
    if 0 in train[col].unique():
        print col
 

N_CASA_MAX_BALANCE_MTD
N_CASA_MIN_BALANCE_MTD
DC_SPEND_MON_01
CR_LIMIT
STMT_MON_03
STMT_MON_02
STMT_MON_01
COC_ELIGIBLE
EOP_BAL_MON_01
AMB_MON_04
AMB_MON_01
AMB_MON_02
AMB_MON_03
LI_MON_04
LI_MON_01
GI_MON_04
GI_MON_01
DC_SPEND_MON_06
DC_SPEND_MON_01_TRL
PL2CC_ELIGIBLE
D_AMT_L3_MON_06
C_AMT_L3_MON_06
C_COUNT_L3_MON_06
D_COUNT_L3_MON_06
D_AMT_L3_MON_05
C_AMT_L3_MON_05
C_COUNT_L3_MON_05
D_COUNT_L3_MON_05
D_AMT_L3_MON_04
C_AMT_L3_MON_04
C_COUNT_L3_MON_04
D_COUNT_L3_MON_04
D_AMT_L3_MON_03
C_AMT_L3_MON_03
C_COUNT_L3_MON_03
D_COUNT_L3_MON_03
D_AMT_L3_MON_02
C_AMT_L3_MON_02
C_COUNT_L3_MON_02
D_COUNT_L3_MON_02
D_AMT_L3_MON_01
C_AMT_L3_MON_01
C_COUNT_L3_MON_01
D_COUNT_L3_MON_01
ZIP_CODE_FINAL
SAL_MON_01
SAL_MON_02
SAL_MON_03
EOP_MON_02
EOP_MON_03
SCRUB_EMI


In [10]:
0.0 in test['DC_SPEND_MON_02'].unique()

False

In [11]:
def addCol(dset, lis3, col_name):
    dset[col_name] = 0.0
    
    for col in lis3:
        dset[col].fillna(0.0, inplace=True)
        
    for col in lis3:
        dset[col_name] += train[col]
        
    dset.drop(lis3, axis=1, inplace=True)

In [12]:
def YNcol(dset, lis2):
    dic = {'Y' :1, 'N' :0}
    for col in lis2: 
        dset[col] = dset[col].map(dic).astype(int)

In [13]:
def justYcol(dset, lis1, col_name):
    dset[col_name] = 0.0
    
    for col in lis1:
        dset[col].fillna(0, inplace=True)
    
    for column in lis1:
        dset[column].replace("Y", 1, inplace=True)
    
    for col in lis1:
        dset[col_name] += dset[col]
        
    dset.drop(lis1, axis=1, inplace=True)

In [14]:
def dataClean(dset):
    dset.drop(['CUSTOMER_ID'],axis=1,inplace=True)
    
    dset = dset[dset.LEGAL_ENTITY != 'FIRM']
    dset = dset[dset.LEGAL_ENTITY != 'ASSOCIATION']
    dset = dset[dset.LEGAL_ENTITY != 'NON_GOVERNMENTAL_ORG']
    dset['LEGAL_ENTITY'] = dset['LEGAL_ENTITY'].map({'INDIVIDUAL':0, 'OTHERS':1}).astype(int)
    
    dset.drop('OCCUP_ALL_NEW', axis=1, inplace=True)
    
    dset['AGE'] = pd.cut(
    dset['AGE'],
    [20, 23, 25, 27, 29, 31, 33, 36, 39, 43, 50, 300],
    labels=[1,2,3,4,5,6,7,8,9,10,11])
    
    dset['N_CASA_BAL_RANGE'] = dset['N_CASA_MAX_BALANCE_MTD'] - dset['N_CASA_MIN_BALANCE_MTD']
    dset.drop(['N_CASA_MAX_BALANCE_MTD','N_CASA_MIN_BALANCE_MTD'], axis=1, inplace=True)
    
    lis = ['BL_TAG','TWL_TAG','PL_TAG','AL_CNC_TAG','AGRI_TAG','CE_TAG','CV_TAG','EDU_TAG','TL_TAG','OTHER_LOANS_TAG','LAP_TAG','LAS_TAG','BBG_TAG','HL_TAG','GL_TAG']
    justYcol(dset, lis, 'LIVE_TAG')
    
    lis = ['AL_CLOSED','PL_CLOSED','BL_CLOSED','TWL_CLOSED','CV_CLOSED','LAP_CLOSED','LAS_CLOSED','BBG_CLOSED','GL_CLOSED']
    justYcol(dset, lis, 'CLOSED_LOAN')
    
    lis = ['CC_HOLD_MON_01','CC_HOLD_MON_02','CC_HOLD_MON_03']
    justYcol(dset, lis, 'CC_HOLD_MON')
    
    lis = ['CC_ACTIVE_MON_01','CC_ACTIVE_MON_02','CC_ACTIVE_MON_03']
    justYcol(dset, lis, 'CC_ACTIVE_MON')
    
    lis = ['DC_ACTIVE_MON_01','DC_ACTIVE_MON_02','DC_ACTIVE_MON_03']
    justYcol(dset, lis, 'DC_ACTIVE_MON')

    lis = ['DC_SPEND_MON_01_ENT', 'DC_SPEND_MON_01_MED', 'DC_SPEND_MON_01_HMD', 'DC_SPEND_MON_01_HBY', 'DC_SPEND_MON_01_CARE', 'DC_SPEND_MON_01_TRL', 'DC_SPEND_MON_01_RST', 'DC_SPEND_MON_01_JER', 'DC_SPEND_MON_01_HTL', 'DC_SPEND_MON_01_ATM']
    addCol(dset, lis, 'DC_SPEND_MON_01_')
    
    lis = ['DC_SPEND_MON_02_ENT', 'DC_SPEND_MON_02_MED', 'DC_SPEND_MON_02_HMD', 'DC_SPEND_MON_02_HBY', 'DC_SPEND_MON_02_CARE', 'DC_SPEND_MON_02_TRL', 'DC_SPEND_MON_02_RST', 'DC_SPEND_MON_02_JER', 'DC_SPEND_MON_02_HTL', 'DC_SPEND_MON_02_ATM']
    addCol(dset, lis, 'DC_SPEND_MON_02_')
  
    lis = ['DC_SPEND_MON_03_ENT', 'DC_SPEND_MON_03_MED', 'DC_SPEND_MON_03_HMD', 'DC_SPEND_MON_03_HBY', 'DC_SPEND_MON_03_CARE', 'DC_SPEND_MON_03_TRL', 'DC_SPEND_MON_03_RST', 'DC_SPEND_MON_03_JER', 'DC_SPEND_MON_03_HTL', 'DC_SPEND_MON_03_ATM']
    addCol(dset, lis, 'DC_SPEND_MON_03_')
    
    lis = ['DC_SPEND_MON_04_ENT', 'DC_SPEND_MON_04_MED', 'DC_SPEND_MON_04_GRC', 'DC_SPEND_MON_04_HMD', 'DC_SPEND_MON_04_HBY', 'DC_SPEND_MON_04_CARE', 'DC_SPEND_MON_04_TRL', 'DC_SPEND_MON_04_RST', 'DC_SPEND_MON_04_JER', 'DC_SPEND_MON_04_HTL', 'DC_SPEND_MON_04_ATM']
    addCol(dset, lis, 'DC_SPEND_MON_04_')
    
    lis = ['DC_SPEND_MON_05_ENT', 'DC_SPEND_MON_05_MED', 'DC_SPEND_MON_05_HMD', 'DC_SPEND_MON_05_HBY', 'DC_SPEND_MON_05_CARE', 'DC_SPEND_MON_05_TRL', 'DC_SPEND_MON_05_RST', 'DC_SPEND_MON_05_JER', 'DC_SPEND_MON_05_HTL', 'DC_SPEND_MON_05_ATM']
    addCol(dset, lis, 'DC_SPEND_MON_05_')

    lis = ['CC_SPEND_MON_01_ENT', 'CC_SPEND_MON_01_MED', 'CC_SPEND_MON_01_HMD', 'CC_SPEND_MON_01_HBY', 'CC_SPEND_MON_01_CARE', 'CC_SPEND_MON_01_TRL', 'CC_SPEND_MON_01_RST', 'CC_SPEND_MON_01_JER', 'CC_SPEND_MON_01_HTL', 'CC_SPEND_MON_01_ATM']
    addCol(dset, lis, 'CC_SPEND_MON_01_')
    
    lis = ['CC_SPEND_MON_02_ENT', 'CC_SPEND_MON_02_MED', 'CC_SPEND_MON_02_HMD', 'CC_SPEND_MON_02_HBY', 'CC_SPEND_MON_02_CARE', 'CC_SPEND_MON_02_TRL', 'CC_SPEND_MON_02_RST', 'CC_SPEND_MON_02_JER', 'CC_SPEND_MON_02_HTL', 'CC_SPEND_MON_02_ATM']
    addCol(dset, lis, 'CC_SPEND_MON_02_')
    
    lis = ['CC_SPEND_MON_03_ENT', 'CC_SPEND_MON_03_MED', 'CC_SPEND_MON_03_HMD', 'CC_SPEND_MON_03_HBY', 'CC_SPEND_MON_03_CARE', 'CC_SPEND_MON_03_TRL', 'CC_SPEND_MON_03_RST', 'CC_SPEND_MON_03_JER', 'CC_SPEND_MON_03_HTL', 'CC_SPEND_MON_03_ATM']
    addCol(dset, lis, 'CC_SPEND_MON_03_')
    
    lis = ['CC_SPEND_MON_04_ENT', 'CC_SPEND_MON_04_MED', 'CC_SPEND_MON_04_HMD', 'CC_SPEND_MON_04_HBY', 'CC_SPEND_MON_04_CARE', 'CC_SPEND_MON_04_TRL', 'CC_SPEND_MON_04_RST', 'CC_SPEND_MON_04_JER', 'CC_SPEND_MON_04_HTL', 'CC_SPEND_MON_04_ATM']
    addCol(dset, lis, 'CC_SPEND_MON_04_')
    
    lis = ['DC_SPEND_MON_01', 'DC_SPEND_MON_02', 'DC_SPEND_MON_03', 'DC_SPEND_MON_04', 'DC_SPEND_MON_05', 'DC_SPEND_MON_06', 'CC_SPEND_MON_01', 'CC_SPEND_MON_02', 'CC_SPEND_MON_03', 'CC_SPEND_MON_04', 'CC_SPEND_MON_05', 'CC_SPEND_MON_06', 'DC_TXN_MON_01', 'DC_TXN_MON_02', 'DC_TXN_MON_03', 'DC_TXN_MON_04', 'DC_TXN_MON_05', 'DC_TXN_MON_06', 'CC_TXN_MON_01', 'CC_TXN_MON_02', 'CC_TXN_MON_03', 'CC_TXN_MON_04', 'CC_TXN_MON_05', 'CC_TXN_MON_06']
    for col in lis:
        dset[col].fillna(0.0, inplace=True)
        
    dset['DC_SPEND_MON'] = dset['DC_SPEND_MON_01'] + dset['DC_SPEND_MON_02'] + dset['DC_SPEND_MON_03'] + dset['DC_SPEND_MON_04'] + dset['DC_SPEND_MON_05'] + dset['DC_SPEND_MON_06']
    dset.drop(['DC_SPEND_MON_01','DC_SPEND_MON_02','DC_SPEND_MON_03','DC_SPEND_MON_04','DC_SPEND_MON_05','DC_SPEND_MON_06'], axis=1, inplace=True)
    
    dset['CC_SPEND_MON'] = dset['CC_SPEND_MON_01'] + dset['CC_SPEND_MON_02'] + dset['CC_SPEND_MON_03'] + dset['CC_SPEND_MON_04'] + dset['CC_SPEND_MON_05'] + dset['CC_SPEND_MON_06']
    dset.drop(['CC_SPEND_MON_01','CC_SPEND_MON_02','CC_SPEND_MON_03','CC_SPEND_MON_04','CC_SPEND_MON_05','CC_SPEND_MON_06'], axis=1, inplace=True)
    
    dset['DC_SPEND_MON_'] = dset['DC_SPEND_MON_01_'] + dset['DC_SPEND_MON_02_'] + dset['DC_SPEND_MON_03_'] + dset['DC_SPEND_MON_04_'] + dset['DC_SPEND_MON_05_'] 
    dset.drop(['DC_SPEND_MON_01_','DC_SPEND_MON_02_','DC_SPEND_MON_03_','DC_SPEND_MON_04_','DC_SPEND_MON_05_'], axis=1, inplace=True)
    
    dset['CC_SPEND_MON_'] = dset['CC_SPEND_MON_01_'] + dset['CC_SPEND_MON_02_'] + dset['CC_SPEND_MON_03_'] + dset['CC_SPEND_MON_04_'] 
    dset.drop(['CC_SPEND_MON_01_','CC_SPEND_MON_02_','CC_SPEND_MON_03_','CC_SPEND_MON_04_'], axis=1, inplace=True)
    
    dset['DC_TXN_MON'] = dset['DC_TXN_MON_01'] + dset['DC_TXN_MON_02'] + dset['DC_TXN_MON_03'] + dset['DC_TXN_MON_04'] + dset['DC_TXN_MON_05'] + dset['DC_TXN_MON_06']  
    dset.drop(['DC_TXN_MON_01','DC_TXN_MON_02','DC_TXN_MON_03','DC_TXN_MON_04','DC_TXN_MON_05', 'DC_TXN_MON_06'], axis=1, inplace=True)
    
    dset['CC_TXN_MON'] = dset['CC_TXN_MON_01'] + dset['CC_TXN_MON_02'] + dset['CC_TXN_MON_03'] + dset['CC_TXN_MON_04'] + dset['CC_TXN_MON_05'] + dset['CC_TXN_MON_06']  
    dset.drop(['CC_TXN_MON_01','CC_TXN_MON_02','CC_TXN_MON_03','CC_TXN_MON_04','CC_TXN_MON_05', 'CC_TXN_MON_06'], axis=1, inplace=True)
    
    dset['PM_FD_MON_01'].fillna(0, inplace=True)

    
    




    
    
    
    
    
    
    
    
    
    
    
    

In [15]:
dataClean(train)

In [17]:
train.head()

,ACT_TYPE,GENDER,LEGAL_ENTITY,OCCUP_ALL_NEW,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,BL_TAG,TWL_TAG,PL_TAG,AL_CNC_TAG,AGRI_TAG,CE_TAG,CV_TAG,EDU_TAG,TL_TAG,OTHER_LOANS_TAG,LAP_TAG,LAS_TAG,BBG_TAG,HL_TAG,GL_TAG,AL_CLOSED,PL_CLOSED,BL_CLOSED,TWL_CLOSED,CV_CLOSED,LAP_CLOSED,LAS_CLOSED,BBG_CLOSED,GL_CLOSED,CC_HOLD_MON_03,CC_HOLD_MON_02,CC_HOLD_MON_01,CC_ACTIVE_MON_03,CC_ACTIVE_MON_02,CC_ACTIVE_MON_01,CC_HOLD,CC_ACTIVE,DC_HOLD,DC_ACTIVE_MON_03,DC_ACTIVE_MON_02,DC_ACTIVE_MON_01,DC_ACTIVE,DC_SPEND_MON_03,DC_SPEND_MON_02,DC_SPEND_MON_01,CC_SPEND_MON_04,CC_SPEND_MON_01,CC_SPEND_MON_02,CC_SPEND_MON_03,DC_SPEND_MON_04,PM_FD_MON_01,CR_LIMIT,STMT_MON_03,STMT_MON_02,STMT_MON_01,COC_ELIGIBLE,LAP_SCRUB_LIVE,LAS_SCRUB_LIVE,PL_SCRUB_LIVE,GL_SCRUB_LIVE,TWL_SCRUB_LIVE,BL_SCRUB_LIVE,CEL_SCRUB_LIVE,TL_SCRUB_LIVE,AL_SCRUB_LIVE,LAP_SCRUB_CLOSED,LAS_SCRUB_CLOSED,PL_SCRUB_CLOSED,GL_SCRUB_CLOSED,TWL_SCRUB_CLOSED,BL_SCRUB_CLOSED,CEL_SCRUB_CLOSED,TL_SCRUB_CLOSED,AL_SCRUB_CLOSED,MATURITY_GL,MATURITY_LAP,MATURITY_LAS,BTE_ACTIVE_MON_01,COC_ACTIVE_MON_01,PL2CC_ACTIVE_MON_01,MER_EMI_ACTIVE_MON_01,TRN_CON_DAE_ACTIVE_MON_01,BTE_CLOSED_MON_01,COC_CLOSED_MON_01,PL2CC_CLOSED_MON_01,TRN_CON_DAE_CLOSED_MON_01,EOP_BAL_MON_01,AMB_MON_04,AMB_MON_01,AMB_MON_02,AMB_MON_03,NB_MON_01_CNT,NB_MON_02_CNT,NB_MON_03_CNT,CDMAT_MON_01,CDMAT_MON_04,MF_MON_04,MF_MON_01,RD_MON_04,RD_MON_01,FD_MON_04,FD_MON_01,LI_MON_04,LI_MON_01,GI_MON_04,GI_MON_01,CC_TXN_MON_01,CC_SPEND_MON_06,CC_TXN_MON_06,CC_TXN_MON_03,CC_TXN_MON_02,CC_TXN_MON_04,CC_SPEND_MON_05,CC_TXN_MON_05,DC_TXN_MON_04,DC_TXN_MON_01,DC_SPEND_MON_06,DC_TXN_MON_06,DC_SPEND_MON_05,DC_TXN_MON_05,DC_TXN_MON_03,DC_TXN_MON_02,DC_SPEND_MON_04_ENT,DC_SPEND_MON_04_MED,DC_SPEND_MON_04_GRC,DC_SPEND_MON_04_HMD,DC_SPEND_MON_04_HBY,DC_SPEND_MON_04_CARE,DC_SPEND_MON_04_TRL,DC_SPEND_MON_04_RST,DC_SPEND_MON_04_JER,DC_SPEND_MON_04_HTL,DC_SPEND_MON_04_ATM,DC_SPEND_MON_03_ENT,DC_SPEND_MON_03_MED,DC_SPEND_MON_03_HMD,DC_SPEND_MON_03_HBY,DC_SPEND_MON_03_CARE,DC_SPEND_MON_03_TRL,DC_SPEND_MON_03_RST,DC_SPEND_MON_03_JER,DC_SPEND_MON_03_HTL,DC_SPEND_MON_03_ATM,DC_SPEND_MON_02_ENT,DC_SPEND_MON_02_MED,DC_SPEND_MON_02_HMD,DC_SPEND_MON_02_HBY,DC_SPEND_MON_02_CARE,DC_SPEND_MON_02_TRL,DC_SPEND_MON_02_RST,DC_SPEND_MON_02_JER,DC_SPEND_MON_02_HTL,DC_SPEND_MON_02_ATM,DC_SPEND_MON_01_ENT,DC_SPEND_MON_01_MED,DC_SPEND_MON_01_HMD,DC_SPEND_MON_01_HBY,DC_SPEND_MON_01_CARE,DC_SPEND_MON_01_TRL,DC_SPEND_MON_01_RST,DC_SPEND_MON_01_JER,DC_SPEND_MON_01_HTL,DC_SPEND_MON_01_ATM,DC_SPEND_MON_05_ENT,DC_SPEND_MON_05_MED,DC_SPEND_MON_05_HMD,DC_SPEND_MON_05_HBY,DC_SPEND_MON_05_CARE,DC_SPEND_MON_05_TRL,DC_SPEND_MON_05_RST,DC_SPEND_MON_05_JER,DC_SPEND_MON_05_HTL,DC_SPEND_MON_05_ATM,CC_SPEND_MON_04_ENT,CC_SPEND_MON_04_MED,CC_SPEND_MON_04_HMD,CC_SPEND_MON_04_HBY,CC_SPEND_MON_04_CARE,CC_SPEND_MON_04_TRL,CC_SPEND_MON_04_RST,CC_SPEND_MON_04_JER,CC_SPEND_MON_04_HTL,CC_SPEND_MON_04_ATM,CC_SPEND_MON_02_ENT,CC_SPEND_MON_02_MED,CC_SPEND_MON_02_HMD,CC_SPEND_MON_02_HBY,CC_SPEND_MON_02_CARE,CC_SPEND_MON_02_TRL,CC_SPEND_MON_02_RST,CC_SPEND_MON_02_JER,CC_SPEND_MON_02_HTL,CC_SPEND_MON_02_ATM,CC_SPEND_MON_01_ENT,CC_SPEND_MON_01_MED,CC_SPEND_MON_01_HMD,CC_SPEND_MON_01_HBY,CC_SPEND_MON_01_CARE,CC_SPEND_MON_01_TRL,CC_SPEND_MON_01_RST,CC_SPEND_MON_01_JER,CC_SPEND_MON_01_HTL,CC_SPEND_MON_01_ATM,FINANCE_MON_01,JOBS_MON_01,LIFESTYLE_MON_01,INMON_01KET_MON_01,TRAVEL_MON_01,CHANNEL_CLICK_DISP,DESIGNATION_FINAL,TOP14_CITY,TOP9_CITY,PL2CC_ELIGIBLE,D_AMT_L3_MON_06,C_AMT_L3_MON_06,C_COUNT_L3_MON_06,D_COUNT_L3_MON_06,MAX_C_AMT_L3_MON_06,D_AMT_L3_MON_05,C_AMT_L3_MON_05,C_COUNT_L3_MON_05,D_COUNT_L3_MON_05,MAX_C_AMT_L3_MON_05,D_AMT_L3_MON_04,C_AMT_L3_MON_04,C_COUNT_L3_MON_04,D_COUNT_L3_MON_04,MAX_C_AMT_L3_MON_04,D_AMT_L3_MON_03,C_AMT_L3_MON_03,C_COUNT_L3_MON_03,D_COUNT_L3_MON_03,MAX_C_AMT_L3_MON_03,D_AMT_L3_MON_02,C_AMT_L3_MON_02,C_COUNT_L3_MON_02,D_COUNT_L3_MON_02,MAX_C_AMT_L3_MON_02,D_AMT_L3_MON_01,C_AMT_L3_MON_01,C_COUNT_L3_MON_01,D_COUNT_L3_MON_01,MAX_C_AMT_L3_MON_01,PA_PQ_TAG,NEFT_CC_AMOUNT,NEFT_CC_TXN,N

In [16]:
dataClean(test)

(300000, 325)